# Deuxième partie : Détection des tumeurs cérébrales avec YOLOv8

Cette partie consiste à classer les images de tumeurs cérébrales et à localiser les tumeurs à l'aide du modèle YOLOv8.

In [ ]:
# Imports et configuration initiale


## 1. Afficher un échantillon d'images pour chaque classe avec les boîtes englobantes

In [ ]:
import os
%pip install Pillow
from PIL import Image
%pip install matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Dossier racine du dataset
data_dir = "../Data/Raw/Data_Brain/Train"

# Récupérer les noms des classes (dossiers)
classes = os.listdir(data_dir)
print("Classes :", classes)

# Fonction pour convertir les coordonnées YOLO en coordonnées d'image
def yolo_to_box(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    box_width = width * img_width
    box_height = height * img_height
    return x_min, y_min, box_width, box_height

# Afficher une image pour chaque classe
plt.figure(figsize=(15, 5))

for i, cls in enumerate(classes):
    img_dir = os.path.join(data_dir, cls, "images")
    label_dir = os.path.join(data_dir, cls, "labels")

    # Prendre la première image et son label correspondant
    img_name = os.listdir(img_dir)[0]
    label_name = os.path.splitext(img_name)[0] + ".txt"

    img_path = os.path.join(img_dir, img_name)
    label_path = os.path.join(label_dir, label_name)

    # Ouvrir l’image
    img = Image.open(img_path).convert("RGB")
    img_w, img_h = img.size

    # Lire le fichier d’annotation
    with open(label_path, "r") as f:
        lines = f.readlines()

    # Afficher l’image
    ax = plt.subplot(1, len(classes), i + 1)
    ax.imshow(img)
    ax.set_title(cls)
    ax.axis("off")

    # Dessiner les boîtes englobantes
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue
        _, x, y, w, h = map(float, parts)
        x_min, y_min, bw, bh = yolo_to_box(x, y, w, h, img_w, img_h)
        rect = patches.Rectangle(
            (x_min, y_min), bw, bh, linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)

plt.tight_layout()
plt.show()


## 3. Filtrer et copier les images et labels

Pour chaque image :
- Vérifier si un fichier .txt correspondant existe dans le répertoire des labels
- Si un label est trouvé : copier l'image et le label vers le dossier approprié (train/valid/test)
- Si aucun label n'est trouvé : afficher un avertissement et sauter l'image

In [ ]:
import os
import shutil

# === 1 Dossiers source et sortie ===
source_dir = "../Data/Raw/Data_Brain"   # Ton dataset
output_dir = "../Data/outputpath"      # Nouveau dossier de sortie

# Créer le dossier de sortie et ses sous-dossiers
for sub in ["images", "labels"]:
    for split in ["Train", "Val"]:
        path = os.path.join(output_dir, sub, split)
        os.makedirs(path, exist_ok=True)

print(" Dossiers créés avec succès !")

# === 2 Extensions d’images acceptées ===
valid_ext = (".jpg", ".jpeg", ".png", ".bmp")

# === 3 Parcourir Train et Val ===
for split in ["Train", "Val"]:
    split_path = os.path.join(source_dir, split)

    print(f"\n🔹 Traitement de {split_path}")

    # Parcourir les classes (Glioma, Meningioma, etc.)
    for cls in os.listdir(split_path):
        class_path = os.path.join(split_path, cls)
        if not os.path.isdir(class_path):
            continue

        img_folder = os.path.join(class_path, "images")
        label_folder = os.path.join(class_path, "labels")

        if not os.path.exists(img_folder) or not os.path.exists(label_folder):
            print(f"⚠️ {cls} n’a pas de dossier images ou labels.")
            continue

        for img_name in os.listdir(img_folder):
            if not img_name.lower().endswith(valid_ext):
                continue

            label_name = os.path.splitext(img_name)[0] + ".txt"
            img_path = os.path.join(img_folder, img_name)
            label_path = os.path.join(label_folder, label_name)

            if os.path.exists(label_path):
                shutil.copy(img_path, os.path.join(output_dir, "images", split, img_name))
                shutil.copy(label_path, os.path.join(output_dir, "labels", split, label_name))
            else:
                print(f"⚠️ Pas de label pour {img_name}")

print("\n✅ Copie terminée avec succès ! Seules les images avec labels ont été copiées.")


## 4. Créer le fichier data.yaml sans augmentations

Ce fichier contient :
- Les chemins des dossiers d'entraînement, de validation et de test
- Le nombre de classes et le nom de chaque classe
- Configuration sans augmentations de données

In [ ]:
%pip install pyyaml
import yaml

data_config = {
    'train': 'Data/outputpath/images/Train',
    'val': 'Data/outputpath/images/Val',
    'nc': 4,
    'names': ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'],
    'augment': False
}

with open('../data.yaml', 'w') as file:
    yaml.dump(data_config, file)

print("✅ Fichier data2.yaml créé avec succès (avec augmentations desactivées) !")


## 5. Créer le fichier data2.yaml avec augmentations

Ce fichier contient la même structure que data.yaml mais avec des augmentations de données activées

In [ ]:
import yaml

data2_config = {
    'train': 'Data/outputpath/images/Train',
    'val': 'Data/outputpath/images/Val',
    'nc': 4,
    'names': ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'],
    'augment': True
}

with open('../data2.yaml', 'w') as file:
    yaml.dump(data2_config, file)

print("✅ Fichier data2.yaml créé avec succès (avec augmentations activées) !")


In [ ]:
import torch
print(torch.cuda.is_available())


In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")


## 6. Compter le nombre d'images et d'étiquettes

Compter les images et labels présents dans les ensembles d'entraînement et de validation

In [ ]:
import os

train_images = '../Data/outputpath/images/Train'
train_labels = '../Data/outputpath/labels/Train'
valid_images = '../Data/outputpath/images/Val'
valid_labels = '../Data/outputpath/labels/Val'

train_img_count = len([f for f in os.listdir(train_images) if f.endswith(('.jpg', '.png', '.jpeg'))])
train_label_count = len([f for f in os.listdir(train_labels) if f.endswith('.txt')])
valid_img_count = len([f for f in os.listdir(valid_images) if f.endswith(('.jpg', '.png', '.jpeg'))])
valid_label_count = len([f for f in os.listdir(valid_labels) if f.endswith('.txt')])

print(f"tain:")
print(f"  - Images: {train_img_count}")
print(f"  - Labels: {train_label_count}")
print(f"\nvalidation")
print(f"  - Images: {valid_img_count}")
print(f"  - Labels: {valid_label_count}")

## 7. Vérifier la correspondance images-labels et nettoyer les données

- Vérifier que chaque image possède un label correspondant
- Supprimer les images sans label
- Supprimer les labels sans image correspondante

In [ ]:
import os

datasets = {
    'train': {
        'images': '../Data/outputpath/images/Train',
        'labels': '../Data/outputpath/labels/Train'
    },
    'valid': {
        'images': '../Data/outputpath/images/Val',
        'labels': '../Data/outputpath/labels/Val'
    }
}

def verify_and_clean(images_dir, labels_dir):
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        print(f"Les répertoires {images_dir} ou {labels_dir} n'existent pas.")
        return
    
    images = {os.path.splitext(f)[0]: f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))}
    labels = {os.path.splitext(f)[0]: f for f in os.listdir(labels_dir) if f.endswith('.txt')}
    
    images_removed = 0
    labels_removed = 0
    
    for img_name in list(images.keys()):
        if img_name not in labels:
            os.remove(os.path.join(images_dir, images[img_name]))
            images_removed += 1
            print(f"Image supprimée (pas de label): {images[img_name]}")
    
    for label_name in list(labels.keys()):
        if label_name not in images:
            os.remove(os.path.join(labels_dir, labels[label_name]))
            labels_removed += 1
            print(f"Label supprimé (pas d'image): {labels[label_name]}")
    
    remaining_count = len(images) - images_removed
    print(f"\nRésumé: {images_removed} images et {labels_removed} labels supprimés")
    print(f"Paires valides restantes: {remaining_count}\n")

for dataset_name, paths in datasets.items():
    print(f"=== Vérification de {dataset_name} ===")
    verify_and_clean(paths['images'], paths['labels'])

## 8. Entraîner le modèle YOLOv8

Lancer l'entraînement du modèle YOLO en définissant les hyperparamètres appropriés

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

model = YOLO('yolov8s.pt') 

model.train(
    data='../data.yaml',      
    epochs=100,             
    batch=-1,
    cos_lr=True,
    patience=10,                       
    device=0,    
    save=True,
    name='model2_yolov8s'          
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt') 

model.train(
    data='../data2.yaml',      
    epochs=100,             
    batch=-1,
    cos_lr=True,
    patience=10,                       
    device=0,    
    save=True,
    name='model_augm'
)

## 9. Évaluer et tester le modèle

Évaluer et tester le modèle après l'entraînement pour mesurer :
- Les performances
- La précision
- La capacité à généraliser sur des données inédites

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path


model_path = 'runs/detect/model2_yolov8s/weights/best.pt' 
model = YOLO(model_path)

print("Évaluation du modèle sur l'ensemble de validation...")

metrics = model.val(data='../data.yaml', split='val')

print("\nMétriques de performance:")
print(f"  - mAP50: {metrics.box.map50:.4f}")
print(f"  - mAP50-95: {metrics.box.map:.4f}")
print(f"  - Précision: {metrics.box.mp:.4f}")
print(f"  - Rappel (Recall): {metrics.box.mr:.4f}")

print("\n Métriques par classe:")
class_names = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
for i, class_name in enumerate(class_names):
    print(f"  {class_name}:")
    print(f"    - Précision: {metrics.box.p[i]:.4f}")
    print(f"    - Rappel: {metrics.box.r[i]:.4f}")
    print(f"    - mAP50: {metrics.box.ap50[i]:.4f}")
    print(f"    - mAP50-95: {metrics.box.ap[i]:.4f}")

print("\nÉvaluation terminée!")


### 9.2 Tester le modèle sur des images de l'ensemble de validation

Visualiser les prédictions du modèle sur des échantillons d'images

In [ ]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

if 'model' not in locals():
    model_path = 'runs/detect/model2_yolov8s/weights/best.pt'
    model = YOLO(model_path)

val_images_dir = '../Data/outputpath/images/Val'

all_images = [f for f in os.listdir(val_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
sample_images = random.sample(all_images, min(6, len(all_images)))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

class_names = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
colors = ['red', 'blue', 'green', 'orange']

for idx, img_name in enumerate(sample_images):
    img_path = os.path.join(val_images_dir, img_name)
    
    results = model.predict(source=img_path, conf=0.25, verbose=False)
    
    img = Image.open(img_path).convert('RGB')
    img_w, img_h = img.size
    
    axes[idx].imshow(img)
    axes[idx].axis('off')
    
    if len(results) > 0 and results[0].boxes is not None:
        boxes = results[0].boxes
        
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy()
            conf = boxes.conf[i].cpu().numpy()
            cls = int(boxes.cls[i].cpu().numpy())
            
            x1, y1, x2, y2 = box
            width = x2 - x1
            height = y2 - y1
            
            rect = patches.Rectangle(
                (x1, y1), width, height,
                linewidth=2,
                edgecolor=colors[cls],
                facecolor='none'
            )
            axes[idx].add_patch(rect)
            
            label = f'{class_names[cls]} {conf:.2f}'
            axes[idx].text(
                x1, y1 - 10,
                label,
                color='white',
                fontsize=10,
                bbox=dict(facecolor=colors[cls], alpha=0.7, edgecolor='none', pad=2)
            )
        
        axes[idx].set_title(f'Image: {img_name}', fontsize=10)
    else:
        axes[idx].set_title(f'Image: {img_name} (Aucune détection)', fontsize=10)

plt.tight_layout()
plt.show()

print("Prédictions affichées!")


### 9.3 Matrice de confusion

Afficher la matrice de confusion pour analyser les performances du modèle par classe

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

confusion_matrix_path = 'runs/detect/model2_yolov8s/confusion_matrix.png'

if os.path.exists(confusion_matrix_path):
    img = Image.open(confusion_matrix_path)
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Matrice de confusion', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    print("Matrice de confusion affichée!")
else:
    print(f"La matrice de confusion n'a pas été trouvée à: {confusion_matrix_path}")
    print("Assurez-vous d'avoir exécuté l'entraînement ou la validation du modèle.")


### 9.4 Courbes de performance (Précision-Rappel et F1)

Afficher les courbes P-R et F1 générées pendant l'entraînement

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

results_dir = 'runs/detect/model2_yolov8s'
plot_files = {
    'Courbe P-R': 'PR_curve.png',
    'Courbe F1': 'F1_curve.png',
    'Courbe P': 'P_curve.png',
    'Courbe R': 'R_curve.png'
}

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, (title, filename) in enumerate(plot_files.items()):
    file_path = os.path.join(results_dir, filename)
    
    if os.path.exists(file_path):
        img = Image.open(file_path)
        axes[idx].imshow(img)
        axes[idx].set_title(title, fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, f'{title}\nFichier non trouvé', 
                      ha='center', va='center', fontsize=12)
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ Courbes de performance affichées!")


### 9.5 Résultats d'entraînement (Loss et mAP)

Afficher l'évolution des métriques pendant l'entraînement

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

results_path = 'runs/detect/model2_yolov8s/results.png'

if os.path.exists(results_path):
    img = Image.open(results_path)
    plt.figure(figsize=(16, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Évolution des métriques pendant l\'entraînement', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    print("Graphique des résultats d'entraînement affiché!")
else:
    print(f"Le fichier des résultats n'a pas été trouvé à: {results_path}")
    print("Assurez-vous d'avoir terminé l'entraînement du modèle.")


## 10. Sauvegarder le modèle entraîné

Sauvegarder le modèle entraîné pour une utilisation future

In [ ]:
from ultralytics import YOLO
import shutil
import os

best_model_path = 'runs/detect/model2_yolov8s/weights/best.pt'
model = YOLO(best_model_path)

output_path = '../models/yolomodel.pt'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
shutil.copy(best_model_path, output_path)

print(f"Modèle sauvegardé avec succès à: {output_path}")
print(f"Taille du fichier: {os.path.getsize(output_path) / (1024*1024):.2f} MB")